## Problem:
The problem is that LWC and NBC are too low by a factor of 2.
The origin of this issue is not yet clear!

## Solution:
simply multiply the scale factor of the parameter dataset by ``2``!!

In [ ]:
import numpy as np
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    label_from_attrs,
    adjust_lightness_array,
    plot_one_one,
    handler_map_alpha,
)
from sdm_eurec4a.reductions import mean_and_stderror_of_mean
from sdm_eurec4a.conversions import msd_from_psd_dataarray
from sdm_eurec4a import RepositoryPath
from sdm_eurec4a.input_processing import models as smodels

default_colors = set_custom_rcParams()
default_dark_colors = adjust_lightness_array(default_colors, 0.75)

RepoPaths = RepositoryPath("levante")
from sdm_eurec4a import slurm_cluster as scluster

In [ ]:
client, cluster = scluster.init_dask_slurm_cluster(
    scale=1,
    processes=16,
    walltime="00:35:00",
    memory="16GB",
    scheduler_options={"dashboard_address": ":8686"},
)

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -e /scratch/m/m301096/dask_logs/dask-worker-%J.err
#SBATCH -o /scratch/m/m301096/dask_logs/dask-worker-%J.out
#SBATCH -p compute
#SBATCH -A mh1126
#SBATCH -n 1
#SBATCH --cpus-per-task=16
#SBATCH --mem=15G
#SBATCH -t 00:35:00

/work/um1487/m301096/conda/envs/sdm_pysd_python312/bin/python -m distributed.cli.dask_worker tcp://10.128.0.218:42257 --name dummy-name --nthreads 1 --memory-limit 0.93GiB --nworkers 16 --nanny --death-timeout 60 --local-directory /scratch/m/m301096/dask_temp --interface ib0



## Data loading

In [ ]:
OBS_data_dir = RepoPaths.data_dir
CLEO_data_dir = RepoPaths.CLEO_data_dir / Path("output_v4.0")

In [ ]:
from sdm_eurec4a.identifications import match_clouds_and_cloudcomposite


cloud_composite = xr.open_dataset(
    OBS_data_dir / Path("observation/cloud_composite/processed/cloud_composite_SI_units_20241025.nc"),
)
identified_clusters = xr.open_dataset(
    OBS_data_dir
    / Path(
        "observation/cloud_composite/processed/identified_clusters/identified_clusters_rain_mask_5.nc"
    )
)
identified_clusters = identified_clusters.swap_dims({"time": "cloud_id"})

# attrs = cloud_composite["radius"].attrs.copy()
# attrs.update({"units": "µm"})
# cloud_composite["radius"] = cloud_composite["radius"]
# cloud_composite["radius_micro"] = 1e6 * cloud_composite["radius"]
# cloud_composite["radius"].attrs = attrs

cloud_composite["radius2D"] = cloud_composite["radius"].expand_dims(time=cloud_composite["time"])
cloud_composite = cloud_composite.transpose("radius", ...)


# cloud_composite = cloud_composite.sel(radius=slice(10e-6, None))

identified_clusters = identified_clusters.where(
    (
        (identified_clusters.duration.dt.seconds >= 3)
        & (identified_clusters.altitude < 1200)
        & (identified_clusters.altitude > 500)
    ),
    drop=True,
)
# identified_clusters = identified_clusters.isel(cloud_id=slice(0, 20))

In [ ]:
list_lwc = []
list_lwc_sem = []
list_lwc_50um = []
list_lwc_50um_sem = []

list_nbc = []
list_nbc_sem = []
list_nbc_50um = []
list_nbc_50um_sem = []
i = 0
N = len(identified_clusters["cloud_id"])
for cloud_id in identified_clusters["cloud_id"]:
    # i += 1
    # print(f"Processing cloud {i}/{N}")
    cc = match_clouds_and_cloudcomposite(
        ds_clouds=identified_clusters.sel(cloud_id=cloud_id),
        ds_cloudcomposite=cloud_composite,
    )
    cc = cc.expand_dims(cloud_id=[int(cloud_id.values)])

    lwc = cc["liquid_water_content"]

    lwc_mean, lwc_sem = mean_and_stderror_of_mean(lwc, dims=("time",))
    list_lwc.append(lwc_mean)
    list_lwc_sem.append(lwc_sem)

    lwc_above_50um = (cc["mass_size_distribution"] * cc["bin_width"]).sel(
        radius=slice(50e-6, None)
    )  # .sum("radius")
    lwc_above_50um_mean, lwc_above_50um_sem = mean_and_stderror_of_mean(lwc_above_50um, dims=("time",))
    list_lwc_50um.append(lwc_above_50um_mean)
    list_lwc_50um_sem.append(lwc_above_50um_sem)

    nbc = cc["particle_size_distribution"] * cc["bin_width"]  # .sum("radius")

    nbc_mean, nbc_sem = mean_and_stderror_of_mean(nbc, dims=("time",))
    list_nbc.append(nbc_mean)
    list_nbc_sem.append(nbc_sem)

    nbc_above_50um = (cc["particle_size_distribution"] * cc["bin_width"]).sel(
        radius=slice(50e-6, None)
    )  # .sum("radius")
    nbc_above_50um_mean, nbc_above_50um_sem = mean_and_stderror_of_mean(nbc_above_50um, dims=("time",))
    list_nbc_50um.append(nbc_above_50um_mean)
    list_nbc_50um_sem.append(nbc_above_50um_sem)


da_lwc = xr.concat(
    list_lwc,
    dim="cloud_id",
)
da_lwc.attrs = dict(
    long_name="Liquid water content",
    units="g m^{-3}",
)

da_lwc_sem = xr.concat(
    list_lwc_sem,
    dim="cloud_id",
)
da_lwc_sem.attrs = dict(
    long_name="Standard error of the mean of the liquid water content",
    units="g m^{-3}",
)

da_lwc_50um = xr.concat(
    list_lwc_50um,
    dim="cloud_id",
)
da_lwc_50um.attrs = dict(
    long_name="Liquid water content above 50 µm",
    units="g m^{-3}",
)

da_lwc_50um_sem = xr.concat(
    list_lwc_50um_sem,
    dim="cloud_id",
)
da_lwc_50um_sem.attrs = dict(
    long_name="Standard error of the mean of the liquid water content above 50 µm",
    units="g m^{-3}",
)

da_nbc = xr.concat(
    list_nbc,
    dim="cloud_id",
)
da_nbc.attrs = dict(
    long_name="Number concentration",
    units="m^{-3}",
)

da_nbc_sem = xr.concat(
    list_nbc_sem,
    dim="cloud_id",
)
da_nbc_sem.attrs = dict(
    long_name="Standard error of the mean of the number concentration",
    units="m^{-3}",
)

da_nbc_50um = xr.concat(
    list_nbc_50um,
    dim="cloud_id",
)
da_nbc_50um.attrs = dict(
    long_name="Number concentration above 50 µm",
    units="m^{-3}",
)

da_nbc_50um_sem = xr.concat(
    list_nbc_50um_sem,
    dim="cloud_id",
)
da_nbc_50um_sem.attrs = dict(
    long_name="Standard error of the mean of the number concentration above 50 µm",
    units="m^{-3}",
)


ds_observations_backup = xr.Dataset(
    dict(
        liquid_water_content=da_lwc,
        liquid_water_content_sem=da_lwc_sem,
        liquid_water_content_50um=da_lwc_50um,
        liquid_water_content_50um_sem=da_lwc_50um_sem,
        particle_size_distribution=da_nbc,
        particle_size_distribution_sem=da_nbc_sem,
        particle_size_distribution_50um=da_nbc_50um,
        particle_size_distribution_50um_sem=da_nbc_50um_sem,
    )
)

In [ ]:
ds_observations = ds_observations_backup
ds_cleo = xr.open_dataset(
    CLEO_data_dir / Path("null_microphysics/combined/eulerian_dataset_combined.nc")
)
ds_cleo = ds_cleo.sel(time=slice(1500, 3590))

ids = np.intersect1d(ds_observations["cloud_id"].values, ds_cleo["cloud_id"].values)
ds_observations, ds_cleo = ds_observations.sel(cloud_id=ids), ds_cleo.sel(cloud_id=ids)

In [ ]:
ds_parameters = xr.open_dataset(
    OBS_data_dir / Path("model/input_v4.0/particle_size_distribution_parameters.nc")
)
ds_parameters = ds_parameters.sel(cloud_id=ids)

ds_parameters_linear = xr.open_dataset(
    OBS_data_dir / Path("model/input_v4.0/particle_size_distribution_parameters_linear_space.nc")
)
ds_parameters_linear = ds_parameters_linear.sel(cloud_id=ids)

In [ ]:
radius = np.geomspace(50e-6, 3e-3, 100)

# radius = np.array([0, 1, 2, 4, 6, 8, 10, 15, 20, 25])

# radius = ds_cleo['radius_bins'].values * 1e-6

t_test = xr.DataArray(
    radius,
    dims="radius",
    coords={"radius": radius},
)
t_test = t_test.expand_dims(cloud_id=ds_cleo["cloud_id"])
w_test = ((t_test["radius"] - t_test["radius"].shift(radius=2)) / 2).shift(radius=-1)
w_test = w_test.interpolate_na("radius", method="linear", fill_value="extrapolate")

# noise = 0.1
# t_test = t_test + noise * np.random.randn(*t_test.shape) * t_test

In [ ]:
# fit the double log-normal distribution
fitted = smodels.double_ln_normal_distribution(
    t=t_test,
    mu1=ds_parameters["mu1"],
    mu2=ds_parameters["mu2"],
    sigma1=ds_parameters["sigma1"],
    sigma2=ds_parameters["sigma2"],
    scale_factor1=ds_parameters["scale_factor1"],
    scale_factor2=ds_parameters["scale_factor2"],
)
fitted2 = smodels.double_ln_normal_distribution(
    t=t_test,
    mu1=ds_parameters["mu1"],
    mu2=ds_parameters["mu2"],
    sigma1=ds_parameters["sigma1"],
    sigma2=ds_parameters["sigma2"],
    scale_factor1=2 * ds_parameters["scale_factor1"],
    scale_factor2=2 * ds_parameters["scale_factor2"],
)

fitted_psd = fitted * w_test
fitted_psd2 = fitted2 * w_test
fitted_msd = msd_from_psd_dataarray(fitted * w_test)
fitted_msd2 = msd_from_psd_dataarray(fitted2 * w_test)

In [ ]:
# fit the double log-normal distribution
fitted_linear = smodels.double_log_normal_distribution_all(
    x=t_test,
    mu1=ds_parameters_linear["geometric_mean1"],
    mu2=ds_parameters_linear["geometric_mean2"],
    sigma1=ds_parameters_linear["geometric_std_dev1"],
    sigma2=ds_parameters_linear["geometric_std_dev2"],
    scale1=ds_parameters_linear["scale_factor1"],
    scale2=ds_parameters_linear["scale_factor2"],
    parameter_space="geometric",
    x_space="linear",
)
fitted_linear2 = smodels.double_log_normal_distribution_all(
    x=t_test,
    mu1=ds_parameters_linear["geometric_mean1"],
    mu2=ds_parameters_linear["geometric_mean2"],
    sigma1=ds_parameters_linear["geometric_std_dev1"],
    sigma2=ds_parameters_linear["geometric_std_dev2"],
    scale1=2 * ds_parameters_linear["scale_factor1"],
    scale2=2 * ds_parameters_linear["scale_factor2"],
    parameter_space="geometric",
    x_space="linear",
)


fitted_linear_psd = fitted_linear * w_test
fitted_linear_psd2 = fitted_linear2 * w_test
fitted_linear_msd = msd_from_psd_dataarray(fitted_linear * w_test)
fitted_linear_msd2 = msd_from_psd_dataarray(fitted_linear2 * w_test)

In [ ]:
x = ds_observations["particle_size_distribution_50um"].sum("radius")
xerr = ds_observations["particle_size_distribution_50um_sem"].sum("radius")
data = (
    ds_cleo["xi"].sel(gridbox=ds_cleo["max_gridbox"]).sel(radius_bins=slice(50, None)).sum("radius_bins")
    / ds_cleo["gridbox_volume"].sel(gridbox=ds_cleo["max_gridbox"])
).compute()
print("done")
y = data.mean("time")
yerr = data.std("time")
y2 = fitted_psd2.sum("radius")
y2err = y2 * 0


ids = np.intersect1d(x.cloud_id.values, y.cloud_id.values)
x, xerr, y, yerr, y2, y2err = (
    x.sel(cloud_id=ids),
    xerr.sel(cloud_id=ids),
    y.sel(cloud_id=ids),
    yerr.sel(cloud_id=ids),
    y2.sel(cloud_id=ids),
    y2err.sel(cloud_id=ids),
)


plt.errorbar(x, y, xerr=xerr, yerr=yerr, fmt="o", color="orange", label="wrong distribution")
plt.errorbar(
    x,
    y2,
    xerr=xerr,
    yerr=y2err,
    fmt="x",
    color="red",
    label="correct distribution\n" + r"2 $\cdot$ scale_factor",
)
plt.plot([0, 12000], [0, 12000], "--", color="black", label="1:1 line")
plt.plot([0, 12000], [0, 6000], "--", color="blue", label="2:1 line")
# plt.xlim(0, 0.7)
# plt.ylim(0, 0.7)
plt.xlabel(label_from_attrs(x))
plt.ylabel("Model")
plt.legend(loc="upper left")

# plt.xscale("log")
# plt.yscale("log")

done


In [ ]:
x = ds_observations["liquid_water_content_50um"]
xerr = ds_observations["liquid_water_content_50um_sem"]
data = ds_cleo["liquid_water_content"].sel(gridbox=ds_cleo["max_gridbox"])
y = data.mean("time")
yerr = data.std("time")
y2 = fitted_msd2.sum("radius")
y2err = y2 * 0


ids = np.intersect1d(x.cloud_id.values, y.cloud_id.values)
x, xerr, y, yerr, y2, y2err = (
    x.sel(cloud_id=ids),
    xerr.sel(cloud_id=ids),
    y.sel(cloud_id=ids),
    yerr.sel(cloud_id=ids),
    y2.sel(cloud_id=ids),
    y2err.sel(cloud_id=ids),
)
x, xerr, y, yerr, y2, y2err = 1e3 * x, 1e3 * xerr, 1e3 * y, 1e3 * yerr, 1e3 * y2, 1e3 * y2err


plt.errorbar(x, y, xerr=xerr, yerr=yerr, fmt="o", color="orange", label="wrong distribution")
plt.errorbar(
    x,
    y2,
    xerr=xerr,
    yerr=y2err,
    fmt="x",
    color="red",
    label="correct distribution\n" + r"2 $\cdot$ scale_factor",
)
plt.plot([0, 2], [0, 2], "--", color="black", label="1:1 line")
plt.plot([0, 2], [0, 1], "--", color="blue", label="2:1 line")
# plt.xlim(0, 0.7)
# plt.ylim(0, 0.7)
plt.xlabel(label_from_attrs(x))
plt.ylabel("Model")
plt.legend(loc="upper left")

# plt.xscale("log")
# plt.yscale("log")

In [ ]:
cleo_psd = (
    (
        ds_cleo["xi"].sel(gridbox=ds_cleo["max_gridbox"]).sel(radius_bins=slice(50, None))
        / ds_cleo["gridbox_volume"].sel(gridbox=ds_cleo["max_gridbox"])
    )
    .mean("time")
    .compute()
)
cleo_psd

In [ ]:
x = ds_observations["particle_size_distribution"] / cloud_composite["bin_width"]
x2 = fitted_psd2 / w_test

cloud_id = np.random.choice(x.cloud_id.values)
plt.plot(1e6 * x.radius, x.sel(cloud_id=cloud_id).T, marker="o", label="OBS")
plt.plot(1e6 * x2.radius, x2.sel(cloud_id=cloud_id).T, label="Correct")
plt.plot(cleo_psd.radius_bins, cleo_psd.sel(cloud_id=cloud_id).T, label="CLEO")
plt.legend()
plt.xscale("log")
plt.yscale("log")